1. Decide What to Monitor

Think about why you’re monitoring before you start:

Authentication events → user logins, logouts, failed logins

Donation actions → amount donated, time, payment method

Browsing actions → pages visited, clicks (if relevant)

Form submissions → donation messages, profile changes

💡 Tip: Avoid storing sensitive data like passwords or full card numbers — Stripe already handles that securely.

2. Structure Your Activity Log

Store it as an array of entries, something like:
[
  {
    "username": "alice",
    "action": "login",
    "timestamp": "2025-08-15T18:23:45Z",
    "details": "Successful login"
  },
  {
    "username": "alice",
    "action": "donation",
    "timestamp": "2025-08-15T18:24:12Z",
    "details": "$20 via Stripe"
  }
]


3. Logging in Your Backend

If you’re already using JSONBin for user data, you can create a second bin just for logs.
Here’s a Flask example:

In [ ]:
import requests
from datetime import datetime

JSONBIN_API_KEY = "your_master_key_here"
LOG_BIN_ID = "your_log_bin_id_here"
BASE_URL = f"https://api.jsonbin.io/v3/b/{LOG_BIN_ID}"
HEADERS = {
    "Content-Type": "application/json",
    "X-Master-Key": JSONBIN_API_KEY
}

def log_activity(username, action, details=""):
    # Get existing logs
    res = requests.get(f"{BASE_URL}/latest", headers=HEADERS)
    logs = res.json().get("record", [])
    
    # Add new log entry
    logs.append({
        "username": username,
        "action": action,
        "timestamp": datetime.utcnow().isoformat(),
        "details": details
    })
    
    # Save back to JSONBin
    requests.put(BASE_URL, headers=HEADERS, json=logs)


Usage:

In [ ]:
log_activity("alice", "login", "Successful login")
log_activity("alice", "donation", "$20 via Stripe")

4. Monitoring Stripe Events

You can log donations automatically using Stripe webhooks:

In [ ]:
if event["type"] == "checkout.session.completed":
    session = event["data"]["object"]
    username = session.get("client_reference_id", "guest")
    amount = session["amount_total"] / 100
    log_activity(username, "donation", f"${amount} via Stripe")


5. Viewing Logs

Since JSONBin is JSON, you can:

Fetch the logs in a dashboard page and format them nicely

Export logs to CSV for analysis

Filter by user, date, or action type

6. Privacy & Compliance

Always inform users you’re logging their activity (privacy policy).

Only store necessary info — e.g., store user IDs instead of full names if possible.

Don’t store IPs or personal identifiers unless required.